In [78]:
import os
import re
import pandas as pd
from sklearn.impute import SimpleImputer
dir_list = os.listdir('../../machine larning/boat_learning/datas')
all_data = []
player_info = []
pattern = '([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s+\d+\s+(\d+.\d{2})\s*\d+\s+(\d+.\d{2})'
pattern_re = re.compile(pattern)
for dir in dir_list:
    if 'K' in dir and 'TXT' in dir:
        with open('../../machine larning/boat_learning/datas/' + dir, encoding='shift-jis') as f:
            data_player = f.readlines()
            for row in data_player:
                row = row.replace('\u3000', '').replace('\n', '')
                if re.match('\s\s0[1-6]', row):
                    all_data.append(row)
for info in all_data:
    player_info.append(info.split()[:4])
column = ['着', '艇番', '選手登番', '選手名']
df1 = pd.DataFrame(player_info, columns=column)
for i, result in enumerate(df1['着']):
    if result == '01' or result =='02':
        df1.loc[i, '着']='0'
    elif result == '03' or result == '04':
        df1.loc[i, '着']='1'
    elif result == '05' or result == '06':
        df1.loc[i, '着']='2'
racers = []
for dir in dir_list:
    if 'B' in dir and 'TXT' in dir:
        with open('../../machine larning/boat_learning/datas/' + dir, encoding='shift-jis') as f:
            data_player = f.readlines()
            for row_program in data_player:
                row_program = row_program.replace('\u3000', '').replace('\n', '')
                if re.match('[1-6]\s', row_program):
                    racers.append(row_program)
values = []
for racer in racers:
    try:
        value = re.match(pattern_re, racer).groups()
        values.append(value)
    except:
        pass
column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
df2 = pd.DataFrame(values, columns=column)
df = pd.merge(df1, df2)
tmp = df.columns
imr = SimpleImputer(missing_values='.', strategy='most_frequent')
imputed_data = imr.fit_transform(df)  
df = pd.DataFrame(imputed_data)
df.columns = tmp

In [79]:
import datetime
import sklearn
import numpy as np
#偏差値変換
list_std = ['全国勝率','全国2連率','当地勝率','当地2連率','モーター2連率','ボート2連率']
for list in list_std:
    df_std = df[list].astype(float)
    win_mean = df_std.mean()
    win_std = df_std.std()
    df[list] = df_std.apply(lambda x : ((x - win_mean)*10/win_std + 50))
list_LabelEncode = ['選手名', '支部', '級別']
le_list = []
for label in list_LabelEncode:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(df[label])
    le_list.append(le)
    df[label] = le.transform(df[label])
df['着'] = df['着'].astype(np.int64)
df['艇番'] = df['艇番'].astype(np.int64)
df['選手登番'] =df['選手登番'].astype(np.int64)
df['年齢'] = df['年齢'].astype(np.int64)
df['体重'] = df['体重'].astype(np.int64)
print('a')

a


In [28]:
cd

C:\Users\shinz


In [81]:
df

,着,艇番,選手登番,選手名,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率
0,0,1,4604,587,33,9,53,0,63.917554,65.267687,62.552014,63.933174,54.738297,52.773066
1,0,1,4604,587,33,9,53,0,63.917554,65.267687,62.552014,63.933174,54.738297,52.773066
2,0,1,4604,587,33,9,53,0,63.917554,65.267687,62.552014,63.933174,54.738297,52.773066
3,0,1,4604,587,33,9,53,0,63.917554,64.902556,59.798770,58.477479,58.979363,46.182496
4,0,1,4604,587,33,9,53,0,63.917554,64.902556,59.798770,58.477479,58.979363,46.182496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400449,2,6,4965,1114,24,2,46,3,44.706649,41.507086,24.873352,32.665529,54.231753,48.705093
2400450,0,2,5163,973,22,5,48,2,38.015211,37.889579,24.873352,32.665529,17.442746,12.922847
2400451,0,2,5163,973,22,5,48,2,38.015211,37.889579,24.873352,32.665529,17.442746,12.922847
2400452,1,4,3176,1351,57,5,51,2,12.112867,27.273715,52.864672,51.648284,68.077292,54.784326


In [52]:
df['全国勝率']

0       39.669959
1       39.669959
2       39.669959
3       39.669959
4       39.669959
          ...    
5867    41.843655
5868    41.843655
5869    48.209480
5870    48.209480
5871     8.539525
Name: 全国勝率, Length: 5872, dtype: float64

In [115]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['着'], axis=1), df['着'], random_state=0)
forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest.fit(X_train, y_train)
print(forest.score(X_train, y_train))
print(forest.score(X_test, y_test))

0.582652165702034
0.5279180289078409


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['着'], axis=1), df['着'], random_state=0)
gbrt = GradientBoostingClassifier(random_state=3, max_depth=2, learning_rate=0.03)
print('a')
gbrt.fit(X_train, y_train)
print(gbrt.score(X_train, y_train))
print(gbrt.score(X_test, y_test))

In [128]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['着'], axis=1), df['着'], random_state=0)
# lgb_clf = lgb.LGBMClassifier(random_state=0)
# lgb_clf.fit(X_train.values, y_train.values)
# y_pred_train = lgb_clf.predict_proba(X_train)[:, 1]
# y_pred = lgb_clf.predict_proba(X_test)[:, 1]
# print(y_pred_train[0])
# print(y_train.values)
# print(accuracy_score(y_train, y_pred_train))
# print(accuracy_score(y_test, y_pred))
train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_test, y_test)
params = {
    # 多値分類問題
    'objective': 'multiclass',
    # クラス数は 3
    'num_class': 3,
    'num_leaves':1000,
}
model = lgb.train(
    params = params,
    train_set = train_set,
    valid_sets = valid_set,
    num_boost_round = 200
)
y_pred_test = model.predict(X_test)
y_pred_test_max = np.argmax(y_pred_test, axis=1)  # 最尤と判断したクラスの値にする
y_pred_train = model.predict(X_train)
y_pred_train_max = np.argmax(y_pred_train, axis=1)
# 精度 (Accuracy) を計算する
print(roc_auc_score(y_train, y_pred_train, multi_class='ovr'))
print(roc_auc_score(y_test, y_pred_test, multi_class='ovr'))
accuracy_test = sum(y_test == y_pred_test_max) / len(y_test)
accuracy_train = sum(y_train == y_pred_train_max) / len(y_train)
print(accuracy_test)
print(accuracy_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.153771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2148
[LightGBM] [Info] Number of data points in the train set: 1800340, number of used features: 13
[LightGBM] [Info] Start training from score -1.029331
[LightGBM] [Info] Start training from score -1.112479
[LightGBM] [Info] Start training from score -1.158326
[1]	valid_0's multi_logloss: 1.06252
[2]	valid_0's multi_logloss: 1.03467
[3]	valid_0's multi_logloss: 1.012
[4]	valid_0's multi_logloss: 0.993378
[5]	valid_0's multi_logloss: 0.977918
[6]	valid_0's multi_logloss: 0.965103
[7]	valid_0's multi_logloss: 0.954443
[8]	valid_0's multi_logloss: 0.945338
[9]	valid_0's multi_logloss: 0.937749
[10]	valid_0's multi_logloss: 0.931214
[11]	valid_0's multi_logloss: 0.925637
[12]	valid_0's multi_logloss: 0.920881
[13]	valid_0's multi_loglo

[197]	valid_0's multi_logloss: 0.906395
[198]	valid_0's multi_logloss: 0.906535
[199]	valid_0's multi_logloss: 0.906657
[200]	valid_0's multi_logloss: 0.906776
0.7793631986864243
0.7459414518805945
0.5513985676054883
0.5858798893542331


In [101]:
0.7763446692396778
0.7546834824938986
0.5603535328287625
0.5828393525667374

980716

In [95]:
y_pred

array([[0.09463254, 0.36177741, 0.54359005],
       [0.16386301, 0.39233094, 0.44380605],
       [0.38693276, 0.37415271, 0.23891453],
       ...,
       [0.08549382, 0.30415761, 0.61034857],
       [0.83240771, 0.11918225, 0.04841004],
       [0.46897568, 0.33750864, 0.19351568]])

In [21]:
pattern = '([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s+\d+\s+(\d+.\d{2})\s*\d+\s+(\d+.\d{2})'
pattern_re = re.compile(pattern)
all_player = []
with open('datas\B220412.TXT', encoding='shift-jis') as f:
    data_player = f.readlines()
    for row_program in data_player:
        row_program = row_program.replace('\u3000', '').replace('\n', '')
        if re.match('[1-6]\s', row_program):
            all_player.append(row_program)
values = []
for player in all_player:
    try:
        value = re.match(pattern_re, player).groups()
        values.append(value)
    except:
        pass
column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
df_player = pd.DataFrame(values)
imr = SimpleImputer(missing_values='.', strategy='most_frequent')
imputed_data = imr.fit_transform(df_player)  
df_player = pd.DataFrame(imputed_data, columns=column)
list_std = ['全国勝率','全国2連率','当地勝率','当地2連率','モーター2連率','ボート2連率']
df_player
for list in list_std:
    df_std = df_player[list].astype(float)
    win_mean = df_std.mean()
    win_std = df_std.std()
    df_player[list] = df_std.apply(lambda x : ((x - win_mean)*10/win_std + 50))
list_LabelEncode = ['選手名', '支部', '級別']
le_list = []
for label in list_LabelEncode:
    le.fit(df_player[label])
    le_list.append(le)
    df_player[label] = le.transform(df_player[label])

In [22]:
player_1 = df_player[12:18]
player_list = []
for i in range(6):
    player_list.append(player_1.iloc[i].to_list())

In [23]:
pred = model.predict(player_list)
print(pred)
print(player_list)

[1.44771189 2.18533411 1.78812539 2.16731635 2.22802027 2.92309564]
[['1', '3334', 435, '53', 7, '45', 1, 47.32159526666452, 46.205656326476195, 51.38743816854713, 52.70092670406486, 55.266133218372424, 51.26670718471153], ['2', '4990', 352, '24', 13, '50', 3, 36.18556712868202, 38.369257664801346, 37.21063401688945, 33.76187033776877, 54.57940094105227, 54.25776202226754], ['3', '4399', 263, '34', 14, '47', 0, 56.90013695178233, 56.18492860448513, 62.4909491125028, 66.92686361463413, 51.74442923211524, 50.353943971372836], ['4', '4525', 369, '33', 13, '46', 2, 43.35000481185957, 45.56764864782656, 48.76026816841476, 48.193431288886394, 57.831107621696404, 52.62883074923234], ['5', '4738', 326, '30', 3, '49', 1, 51.8383059799721, 53.33469864877862, 43.1093742058659, 37.143844688893076, 56.88024754540697, 52.08117282122913], ['6', '5203', 383, '17', 13, '46', 3, 21.000074213251345, 26.566115609783132, 31.80758628076817, 33.76187033776877, 49.57271177392331, 53.766274138162096]]


In [67]:
import os
import re
import pandas as pd
from sklearn.impute import SimpleImputer
dir_list = os.listdir('datas')
all_data = []
player_info = []
pattern = '([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s+\d+\s+(\d+.\d{2})\s*\d+\s+(\d+.\d{2})'
pattern_re = re.compile(pattern)
for dir in dir_list:
    if 'K' in dir and 'TXT' in dir:
        with open('datas\\' + dir, encoding='shift-jis') as f:
            data_player = f.readlines()
            for row in data_player:
                row = row.replace('\u3000', '').replace('\n', '')
                if re.match('\s\s0[1-6]', row):
                    all_data.append(row)
for info in all_data:
    player_info.append(info.split()[:4])
column = ['着', '艇番', '選手登番', '選手名']
df1 = pd.DataFrame(player_info, columns=column)
for i, result in enumerate(df1['着']):
    if result == '01' or result =='02':
        df1.loc[i, '着']=1
    elif result == '03' or result == '04':
        df1.loc[i, '着']=2
    elif result == '05' or result == '06':
        df1.loc[i, '着']=3
racers = []
for dir in dir_list:
    if 'B' in dir and 'TXT' in dir:
        with open('datas\\' + dir, encoding='shift-jis') as f:
            data_player = f.readlines()
            for row_program in data_player:
                row_program = row_program.replace('\u3000', '').replace('\n', '')
                if re.match('[1-6]\s', row_program):
                    racers.append(row_program)
values = []
for racer in racers:
    try:
        value = re.match(pattern_re, racer).groups()
        values.append(value)
    except:
        pass
column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
df2 = pd.DataFrame(values, columns=column)
df = pd.merge(df1, df2)
tmp = df.columns
imr = SimpleImputer(missing_values='.', strategy='most_frequent')
imputed_data = imr.fit_transform(df)  
df = pd.DataFrame(imputed_data)
df.columns = tmp

In [ ]:
import datetime
import sklearn
#偏差値変換
list_std = ['全国勝率','全国2連率','当地勝率','当地2連率','モーター2連率','ボート2連率']
for list in list_std:
    df_std = df[list].astype(float)
    win_mean = df_std.mean()
    win_std = df_std.std()
    df[list] = df_std.apply(lambda x : ((x - win_mean)*10/win_std + 50))
list_LabelEncode = ['選手名', '支部', '級別']
le_list = []
for label in list_LabelEncode:
    le = sklearn.preprocessing.LabelEncoder()
    le.fit(df[label])
    le_list.append(le)
    df[label] = le.transform(df[label])
print('a')

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['着'], axis=1), df['着'], random_state=0)
forest = RandomForestClassifier(n_estimators=5, min_samples_split=20, random_state=2)
forest.fit(X_train, y_train)
print(forest.score(X_train, y_train))
print(forest.score(X_test, y_test))

0.7723577235772358
0.5403225806451613


In [69]:
df

,着,艇番,選手登番,選手名,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率
0,1,1,3906,飯山晃三,44,長崎,57,B1,5.32,35.24,5.61,38.01,25.90,28.28
1,1,1,3906,飯山晃三,44,長崎,58,B1,5.37,35.71,5.06,27.78,33.33,28.57
2,1,1,3906,飯山晃三,44,長崎,58,B1,5.25,34.75,5.12,30.61,35.82,41.74
3,1,1,3906,飯山晃三,44,長崎,59,B1,5.21,34.62,5.79,47.37,22.86,0.00
4,1,1,3906,飯山晃三,44,長崎,58,B1,5.06,33.04,5.24,41.18,25.00,39.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409367,2,6,3176,長谷川巌,57,山口,51,B1,0.00,0.00,5.49,34.69,50.98,36.84
409368,3,4,5210,藤井公人,23,福岡,57,B2,1.18,0.00,1.33,0.00,55.17,22.22
409369,3,3,3176,長谷川巌,57,山口,51,B1,0.00,0.00,5.49,34.69,50.98,36.84
409370,2,4,3176,長谷川巌,57,山口,51,B1,0.00,0.00,5.49,34.69,50.98,36.84


In [169]:
!pip install lightgbm


     ---------------------------------------- 1.0/1.0 MB 2.9 MB/s eta 0:00:00


In [38]:
import lightgbm as lgb
train_set = lgb.Dataset(X_train, y_train)
test_set = lgb.Dataset(X_test, y_test)
params = {
    "objective" : "regression",
    "metric" : "rmse"
}
model = lgb.train(
    params = params,
    train_set = train_set,
    valid_sets = [train_set, test_set],
    num_boost_round = 100
)
pred = model.predict(X_test)
print(pred)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: 艇番, 選手登番, モーター, ボート, 展示タイム, 進入, スタートタイミング, レースタイム, 年齢, 体重

In [39]:
X_train

,艇番,選手登番,選手名,モーター,ボート,展示タイム,進入,スタートタイミング,レースタイム,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率
406,4,4891,81,63,81,6.75,4,0.17,109.8,32,10,47,2,38.962398,42.896115,43.734155,41.339263,55.230883,48.910333
179,2,4553,50,22,74,6.62,2,0.07,108.5,32,15,54,1,53.456539,46.203853,54.543407,53.704190,45.660066,36.469988
478,6,3141,107,31,38,6.70,6,0.17,109.8,60,10,56,2,42.282285,42.472608,46.966333,46.345306,26.674170,44.068454
65,4,4057,154,55,15,6.74,3,0.14,110.9,43,2,52,1,54.023349,53.929209,56.080016,54.494033,61.049827,53.452273
467,4,5147,214,34,16,6.83,4,0.16,109.8,28,3,53,3,22.201018,27.284762,33.030875,33.101540,49.808364,53.423707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,5,4625,209,1,46,6.63,5,0.12,109.8,36,16,52,2,41.391584,43.881865,50.781363,50.695002,59.464129,48.810353
192,6,4270,116,23,71,6.71,6,0.17,109.8,39,0,64,2,39.367263,40.617938,48.926835,46.211812,43.196572,54.309243
117,4,3978,295,65,63,6.93,4,0.18,112.3,44,10,52,0,64.063983,61.691075,24.500044,33.101540,50.374685,51.866879
47,1,3661,162,37,73,6.70,1,0.11,108.3,52,5,59,2,51.756109,47.050868,50.834350,45.355222,45.207010,46.910737


In [29]:
feature = forest.feature_importances_
print(feature)
print(df.columns)

[0.14462229 0.04626049 0.04432345 0.17127397 0.27955347 0.03419157
 0.03312497 0.02741185 0.01132889 0.05475783 0.04232068 0.02912281
 0.02601736 0.02762749 0.02806288]
Index(['着', '艇番', '選手登番', '選手名', 'モーター', '展示タイム', '年齢', '支部', '体重', '級別',
       '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率'],
      dtype='object')


In [194]:
pattern = '([1-6])\s(\d{4})([^0-9]+)(\d{2})([^0-9]+)(\d{2})([AB]\d{1})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s(\d{1}.\d{2})\s*(\d+.\d{2})\s+\d+\s+(\d+.\d{2})\s*\d+\s+(\d+.\d{2})'
pattern_re = re.compile(pattern)
all_player = []
with open('datas\B220405.TXT', encoding='shift-jis') as f:
    for row_program in data_player:
        row_program = row_program.replace('\u3000', '').replace('\n', '')
        if re.match('[1-6]\s', row_program):
            all_player.append(row_program)
values = []
for player in all_player:
    try:
        value = re.match(pattern_re, player).groups()
        values.append(value)
    except:
        pass
column = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']
df_player = pd.DataFrame(values)
imr = SimpleImputer(missing_values='.', strategy='most_frequent')
imputed_data = imr.fit_transform(df_player)  
df_player = pd.DataFrame(imputed_data, columns=column)
list_std = ['全国勝率','全国2連率','当地勝率','当地2連率','モーター2連率','ボート2連率']
df_player
for list in list_std:
    df_std = df_player[list].astype(float)
    win_mean = df_std.mean()
    win_std = df_std.std()
    df_player[list] = df_std.apply(lambda x : ((x - win_mean)*10/win_std + 50))
list_LabelEncode = ['選手名', '支部', '級別']
le_list = []
for label in list_LabelEncode:
    le.fit(df_player[label])
    le_list.append(le)
    df_player[label] = le.transform(df_player[label])

In [217]:
player_1 = df_player[30:36]
player_list = []
for i in range(6):
    player_list.append(player_1.iloc[i].to_list())

In [218]:
forest.predict(player_list)

array(['01', '02', '01', '03', '06', '06'], dtype=object)

In [160]:
# list_LabelEncode = ['選手名', '支部', '級別']
# for i, label in enumerate(list_LabelEncode):
#     try:
#         le_list[i].fit(df_player[label])
#         df_player[label] = le.transform(df_player[label])
#     except ValueError as v:
#         df_player[label] = 0
# forest.predict(['1', '3931', '黒崎竜也', '', '', '45', '三重' '50', 'A2', '6.33', '49.28', '0.00', '0.00', '28.44', '22.22'])

In [132]:
le_list[0]

LabelEncoder()

In [158]:
df_player.columns = ['艇番', '選手登番', '選手名', '年齢', '支部', '体重', '級別', '全国勝率', '全国2連率', '当地勝率', '当地2連率', 'モーター2連率', 'ボート2連率']

In [114]:
info.split()[:4]

['06', '2', '4931', '木谷賢太']

In [166]:
forest.predict(df_player)

array(['05'], dtype=object)

In [216]:
df_player[18:24]

,艇番,選手登番,選手名,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率
18,1,4066,284,42,0,52,1,56.962031,58.701146,25.999522,33.616562,57.035350,39.616141
19,2,3949,48,45,6,57,2,42.080444,38.109877,48.654963,46.046655,59.038406,37.350930
20,3,3935,228,44,6,55,2,47.066154,46.139704,50.651037,48.577912,47.076835,29.436942
21,4,3873,73,45,7,53,0,52.731732,51.935144,59.333959,58.826814,46.468620,29.436942
22,5,4423,116,32,6,56,2,46.990613,45.071388,53.445541,60.544838,40.832489,29.436942
23,6,4607,524,33,13,53,1,45.404251,43.633001,52.297798,49.283433,46.663249,29.436942


In [164]:
df

,着,艇番,選手登番,選手名,年齢,支部,体重,級別,全国勝率,全国2連率,当地勝率,当地2連率,モーター2連率,ボート2連率
0,01,1,3906,1341,44,15,57,2,50.067933,50.862951,53.461115,53.366389,41.477508,45.432614
1,01,1,3906,1341,44,15,58,2,50.432500,51.182779,50.717045,47.893904,49.864712,45.761370
2,01,1,3906,1341,44,15,58,2,49.557540,50.529514,51.016398,49.407798,52.675497,60.691449
3,01,1,3906,1341,44,15,59,2,49.265886,50.441051,54.359174,58.373471,38.045866,13.373190
4,01,1,3906,1341,44,15,58,2,48.172186,49.365886,51.615104,55.062163,40.461562,58.016051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404012,06,1,5030,1282,24,14,44,3,29.068881,32.884557,33.903378,33.033139,12.240823,13.373190
404013,04,6,3176,1322,57,5,51,2,11.278018,26.882686,52.862409,51.590372,69.788553,55.136598
404014,06,4,5210,1181,23,13,57,3,19.881796,26.882686,32.107259,33.033139,74.518349,38.562737
404015,06,3,3176,1322,57,5,51,2,11.278018,26.882686,52.862409,51.590372,69.788553,55.136598
